# Simple Retrieval Augmented Generation (RAG)

This notebook will show you how to build a simple RAG application with Llama Stack. You will learn how the API's provided by Llama Stack can be used to directly control and invoke all common RAG stages, including indexing, retrieval and inference.

## Overview

This tutorial covers the following steps:
1. Setting up the Llama Stack client
2. Creating and reusing an agent for a conversation like approach with our LLM
3. Indexing a collection of documents into a vector database for later retrieval.
4. Executing the built-in RAG tool to retrieve the document chunks relevant to a given query.
5. Using the retrieved context to answer user queries during the inference step.

## 1. Setting Up this Notebook

First of all, we need to install the Llama Stack client which we will use for this use case. The Llama Stack server has already been deployed as part of OpenShift AI so we only need to get the client and connect as well as use the provided API's later: 

In [ ]:
%pip install llama_stack_client

Then, we will need a few imports we use throughout the notebook:

In [ ]:
from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger

from uuid import uuid4

import io
import requests

Now we initialize the Llama Stack client itself. We only need to provide the endpoint it's reachable with. This is exposed via a Kubernetes Service on the underlying cluster:

In [ ]:
client = LlamaStackClient(base_url='http://lsd-llama-milvus-service.llamastack.svc.cluster.local:8321')

print('Connected to Llama Stack server')

Next, we want to check which models are available to us:

In [ ]:
# Fetch all registered models
models = client.models.list()

# Let's see what's in there
print('Following models are available:')
print(models)

Taking a look at the `model_type` of our models, we can see that we have an LLM available (`llama-3-2-3b`) as well as two embedding models. The LLM is used for the actual inferencing later, digesting your queries / messages and answering based on trained information and later even with additional information you provided via RAG.

For now let's set the model vars for our LLM and embedding model for later usage:

In [ ]:
# We select the first llm available model - in this case we only have one
model_id = next(m.identifier for m in models if m.model_type == "llm")

# Also selecting the first available embedding model, we have 2 but any works
embedding_model = next(m for m in models if m.model_type == "embedding")
embedding_model_id = embedding_model.identifier

A quick check to see if we have set the correct models, should be `vllm-inference/llama-3-2-3b` and `granite-embedding-125m`:

In [ ]:
print(model_id)
print(embedding_model_id)

Great so far! Let's check now what kind of providers are available from our Llama Stack server. These providers are basically capabilities which are (partially) implemented:

In [ ]:
for provider in client.providers.list():
    print(provider.provider_id)

There are quite some, right?! Some are self explanatory, like `llm-inference` which is the basic capability of querying the configured LLM or `sentence-transformers` (which is used by the embedding model) to digest text for our actual RAG use case. The Llama Stack can do quite some stuff, but we will focus on the basics for now. If you are eager to do more, feel free to reach out to us!

An important provider is `milvus`. This is our vector database of choice here, as it is integrated into the server side Llama Stack via OpenShift AI. This means we can already interact with a vector database to store and _*retrieve*_ (as in RAG). We only need to create our own store in the database and use it with our Llama Stack client. To do so:

In [ ]:
# Your unique ID for your own vector store containing documents for the RAG use case
vector_store_name = f'vector_store_{uuid4()}'

# Set the milvus id, which is the one listed in our earlier step
provider_id = 'milvus'

# Initialize and set our own store
vector_store = client.vector_stores.create(
    name=vector_store_name,
    extra_body={
        "provider_id": provider_id,
        "embedding_model": embedding_model_id,
    },
)

print(f"Registered vector Store: {vector_store_name}")

So let's check now what your vector store looks like:

In [ ]:
print(vector_store)

You should see a section `FileCounts( ... )` where you can see, that there no files have been ingested so far (I mean that makes sense since we literally just created the store). Let's check that later once we ingested something!

### Status

Congrats! If you made it this far, you should have your own vector store instance in the Milvus VectorDB and configured your Llama Stack client and Jupyter notebook environment to leverage the necessary basics:
- An LLM for the inferencing part
- An embedding model for the ingestion of documents
- Your own vector store to ingest your documents for retrieval
- The Llama Stack client itself to do all that with simplified functions

In the next step we will start to query the LLM via the Llama Stack and prepare the rest for the actual retrieval part!

## 2. Let's get querying
Now comes the fun part: we start to interact the LLM through the Llama Stack. The underlying LLM is quite small and has limited information stored. We will start by preparing everything to be able to interact with an agent to ultimately talk with our LLM. Once we did that, in step 3 we will start adding new information for the actual RAG use case.

So let's get going!

First of all, let's check the capabilities our Llama Stack server has:

In [ ]:
client.tools.list()

Our stack comes with 3 built-in tools: `web_search`, `insert_into_memory` and `knowledge_search`, let's drill them down:
- `web_search`: as the name indicates, triggers a web search by the Llama Stack server to collect additional information for a given request
- `insert_into_memory`: allows for ephemeral knowledge, is for example used internally after doing a web search to put this information in there for a session to serve as a form of memory
- `knowledge_search`: will retrieve information available in provided vector stores - we will use this capability for our use case here

To interact with our LLM through the Llama Stack client, we can leverage two API's, the `Response` and the `Agent` one. The `Response` is perfect for "one off" / stateless queries where we just want to query once while `Agent` is a great wrapper to maintain a stateful conversation. Let's go with the `Agent` one:

In [ ]:
# This is the list of tools we want to leverage in our agent.
# We can make use of the tools provided out of the box by Llama Stack
# (as seen by the list tools call before) and also custom ones.
# Here we go with the file search tool which is the one to search
# in vector stores (our RAG knowledge base in this case).
#
# NOTE: you might be wondering why we call it file_search and not
# knowledge_search as mentioned before. The reason is that the Llama Stack
# API maintains OpenAI compatibility where it's called like this and
# it will be mapped accordingly on the server side.
tools = [
    {
        "type": "file_search",
        "vector_store_ids": [vector_store.id],
    }
]

# These instructions are very important to guide the agent behavior
# towards the desired RAG use case. Here we explicitly tell it to
# ALWAYS use the knowledge_search tool. We do so by providing clear
# instructions on how to behave - that's just how LLMs work best!
instructions = """You are a helpful AI assistant with access to a knowledge base via the knowledge_search tool
When answering questions:
1. ALWAYS use the knowledge_search tool first
2. Provide specific details from the documentation
3. If information isn't in the knowledge base, say so clearly
4. Be concise but thorough
"""

agent = Agent(
    client=client,
    model=model_id,
    instructions=instructions,
    tools=tools,
)

With the agent instantiated, we can use it to have a conversation now! As stated before, it is stateful, the statefulness comes from a unique session ID we provide. Every subsequent invocation with this ID will leverage this.

*Important: if you want to start a new conversation rerun the session ID generation here to create a new unique one to use!*

In [ ]:
session_id = agent.create_session(f"interactive-{uuid4().hex[:8]}")

Now your actual message you want to provide to the Llama Stack / model and the callback to our agent to start our conversation:

In [ ]:
user_input = "How are you doing?"

messages = [
    {
        "type": "message",
        "role": "user",
        "content": [{"type": "input_text", "text": user_input}],
    }
]

print("🤖 Assistant:", end=" ", flush=True)

# Stream response with event logging
event_logger = AgentEventLogger()

for chunk in agent.create_turn(messages=messages, session_id=session_id, stream=True):
    for log_msg in event_logger.log([chunk]):
        print(log_msg, end="", flush=True)

As you can see from the output, the `file_search` tool has been used for this query. For the message / question we used this was not really necessary as it added no new knowledge (and we don't even have any documents to retrieve so far) but we stated in the instructions to *ALWAYS* use the `file_search` first, so it's performing as intended! You can already play around with the message above and see what the responses are.

### Status
Now we are very close for the actual RAG part. We have a working agent to talk to in a stateful, conversation like manner and learnt a bit about the API and the possibilities we have. So far we haven't ingested any new knowledge so it's answering based on the data it has from the training of the model itself. The ingestion is not that hard - let's get going!

## 3. Ingesting and retrieving knowledge
The last step of this guided lab exercise is to ingest and retrieve knowledge together, so we finally do the RAG bit of this whole exercise.

Let's start by asking the agent a question it does not really have knowledge about (it's a weird one but it will make sense later):

In [ ]:
user_input = "What is the size of my pretzels?"

messages = [
    {
        "type": "message",
        "role": "user",
        "content": [{"type": "input_text", "text": user_input}],
    }
]

print("🤖 Assistant:", end=" ", flush=True)

event_logger = AgentEventLogger()

for chunk in agent.create_turn(messages=messages, session_id=session_id, stream=True):
    for log_msg in event_logger.log([chunk]):
        print(log_msg, end="", flush=True)

As you can see, it does not know anything about any sizing of pretzels - this is good! Well, at least for us to ingest that information now!

Let's create a document based on a small, raw text sample, wrap it as a file and ingest it into our vector store:

In [ ]:
# Example text we wrap into a document to ingest into our vector store for later retrieval
#
# (I just love pretzels, don't judge me)
raw_text = """
Size of my pretzels are 30cm.
"""

# Create a pseudo file object from the raw text as the API expects a file-like object
pseudo_file = io.BytesIO(str(raw_text).encode('utf-8'))
file_id = client.files.create(file=('raw_text.txt', pseudo_file, "text/plain"), purpose="assistants").id

# Ingest the file into the vector store
vector_store_file = client.vector_stores.files.create(vector_store_id=vector_store.id, file_id=file_id)

print(vector_store_file.status)

Now let's ask it again:

In [ ]:
user_input = "What is the size of my pretzels?"

messages = [
    {
        "type": "message",
        "role": "user",
        "content": [{"type": "input_text", "text": user_input}],
    }
]

print("🤖 Assistant:", end=" ", flush=True)

event_logger = AgentEventLogger()

for chunk in agent.create_turn(messages=messages, session_id=session_id, stream=True):
    for log_msg in event_logger.log([chunk]):
        print(log_msg, end="", flush=True)

And that's how easy it is - with the configuration we have done previously and the capability of ingesting documents, we are able to augment the knowledge of the LLM by practically anything we want.

Here an example of ingesting the content of a website:

*NOTE: the lab environment does not have a GPU, embedding is done locally with the CPU and will take some time!*

In [ ]:
# Take a look at the page itself, it mainly contains text so it's a good candidate for ingestion
url = "https://www.paulgraham.com/greatwork.html"

# We basically just fetch the content and wrap it as a file-like object for ingestion
# just like we did with the raw text before
response = requests.get(url)
pseudo_file = io.BytesIO(str(response.content).encode('utf-8'))
file_id = client.files.create(file=('paulgraham-greatwork.html', pseudo_file, "text/html"), purpose="assistants").id

# Ingest the file into the vector store
vector_store_file = client.vector_stores.files.create(vector_store_id=vector_store.id, file_id=file_id)

print(vector_store_file.status)

Now let's see what output we will get:

In [ ]:
user_input = "Is there a file with content about Paul Graham in the knowledge base?"

messages = [
    {
        "type": "message",
        "role": "user",
        "content": [{"type": "input_text", "text": user_input}],
    }
]

print("🤖 Assistant:", end=" ", flush=True)

event_logger = AgentEventLogger()

for chunk in agent.create_turn(messages=messages, session_id=session_id, stream=True):
    for log_msg in event_logger.log([chunk]):
        print(log_msg, end="", flush=True)

### Status
Well done! By now you should have an environment capable of chatting with an agent and expanding it's knowledge by ingesting any kind of content you want. You need some base configuration and snippets for the ingestion part but once this is done, building on top of that is a piece of cake - that's how RAG is supposed to be :)

Since the Llama Stack is part of OpenShift AI, getting this environment set up like this is very straight forward. We only have to enable the Llama Stack as part of the OpenShift AI operator and you are already able to build notebooks like this. Any Llama Stack client is feasible so you can build any application you need based on this stack!

## (Optional) Sandbox time!
If you made it this far and you still have some time, feel free to use this part of the notebook to add your own code based on the snippets above to play around!

---

---

## Key Takeaways
This notebook demonstrated how to set up and use the built-in RAG tool for ingesting user-provided documents in a vector database and utilizing them during inference via direct retrieval. 

I hope you learned something and enjoyed this lab! If you have any questions or want to try it out in your environment, feel free to get in touch with a Red Hatter of your choice! :)